In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, log_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy.special import softmax
import json
#from ucimlrepo import fetch_ucirepo

# === User configuration ===
TRAIN_PATH = ""  # Path to your training data file
TEST_PATH  = ""   # Path to your test data file
TARGET_COLUMN = 'satisfaction'                    # Name of the target column, or None to auto-detect
VAL_SIZE = 0.25                            # Fraction of TRAIN data to reserve for validation
N_ESTIMATORS = 100                          # Number of trees in each Random Forest
RANDOM_STATE = 42                          # Random seed for reproducibility
TEMPERATURE = 1.0                          # Softmax temperature (<1 -> sharper; >1 -> smoother)
COMPOSITE_WEIGHTS = [1.0, 1.0, 1.0, 1.0]    # [accuracy, precision, recall, f1]


def load_data(file_path):
    """
    Load dataset by file extension. Supports .csv and .data as CSVs.
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext in ['.csv', '.data']:
        return pd.read_csv(file_path)
    raise ValueError(f"Unsupported file extension: {ext}")


def detect_target_column(df):
    """
    Auto-detect target column: prefers 'target', then 'class', otherwise last column.
    """
    if 'target' in df.columns:
        return 'target'
    if 'class' in df.columns:
        return 'class'
    return df.columns[-1]


def evaluate_random_forests_explicit(df_train, df_test, target_column,
                                    val_size=VAL_SIZE, n_estimators=N_ESTIMATORS,
                                    random_state=RANDOM_STATE, temperature=TEMPERATURE,
                                    composite_weights=COMPOSITE_WEIGHTS):
    """
    Trains one Random Forest on df_train (optionally splitting off validation),
    evaluates on df_test, for both standard majority-vote RF and composite-metric softmax RF.
    Returns metrics, log-losses, composite scores, weights, processed splits, label encoder, RF, and preprocessor.
    """
    # Split out features & labels
    X_train_full = df_train.drop(columns=[target_column])
    y_train_full = df_train[target_column]
    X_test = df_test.drop(columns=[target_column])
    y_test = df_test[target_column]

    # Encode labels to integers
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train_full)
    y_test_enc  = le.transform(y_test)

    # Optionally split train into train + validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_enc, test_size=val_size,
        random_state=random_state, stratify=y_train_enc
    )

    # Identify categorical and numerical columns
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
    numerical_cols   = X_train.select_dtypes(include=np.number).columns

    # Build preprocessing pipeline
    preprocessor = ColumnTransformer([
        ('num', 'passthrough', numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ], remainder='passthrough')

    # Fit on train, transform train/val/test
    X_train_proc = preprocessor.fit_transform(X_train)
    X_val_proc   = preprocessor.transform(X_val)
    X_test_proc  = preprocessor.transform(X_test)

    # Train a single Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X_train_proc, y_train)

    # --- Standard RF evaluation ---
    proba_std = rf.predict_proba(X_test_proc)
    log_std = log_loss(y_test_enc, proba_std)
    y_pred_std = np.argmax(proba_std, axis=1)
    metrics_standard = classification_report(
        le.inverse_transform(y_test_enc),
        le.inverse_transform(y_pred_std),
        output_dict=True
    )

    # --- Composite-metric softmax RF ---
    # Compute per-tree metrics on validation set
    tree_acc, tree_prec, tree_rec, tree_f1 = [], [], [], []
    for tree in rf.estimators_:
        preds_val = tree.predict(X_val_proc)
        tree_acc.append(accuracy_score(y_val, preds_val))
        tree_prec.append(precision_score(y_val, preds_val, average='weighted', zero_division=0))
        tree_rec.append(recall_score(y_val, preds_val, average='weighted', zero_division=0))
        tree_f1.append(f1_score(y_val, preds_val, average='weighted', zero_division=0))

    metrics_matrix = np.vstack([tree_acc, tree_prec, tree_rec, tree_f1]).T
    composite_scores = metrics_matrix.dot(np.array(composite_weights))
    weights = softmax(composite_scores / temperature)

    # Aggregate weighted probabilities
    proba_weighted = np.zeros_like(proba_std)
    for w, tree in zip(weights, rf.estimators_):
        proba_weighted += w * tree.predict_proba(X_test_proc)

    log_weighted = log_loss(y_test_enc, proba_weighted)
    y_pred_w = np.argmax(proba_weighted, axis=1)
    metrics_weighted = classification_report(
        le.inverse_transform(y_test_enc),
        le.inverse_transform(y_pred_w),
        output_dict=True
    )

    return (metrics_standard, metrics_weighted,
            log_std, log_weighted,
            composite_scores, weights,
            X_train_proc, y_train, X_test_proc, y_test_enc,
            le, rf, preprocessor)


def print_summary(metrics, title, logloss=None):
    """
    Prints overall accuracy, weighted-average precision, recall, F1, and optional log-loss.
    """
    accuracy = metrics.get('accuracy')
    weighted = metrics.get('weighted avg', {})
    precision = weighted.get('precision')
    recall = weighted.get('recall')
    f1 = weighted.get('f1-score')

    print(f"\n{title} Summary:")
    if accuracy is not None:
        print(f"  Accuracy : {accuracy:.4f}")
    if precision is not None:
        print(f"  Precision: {precision:.4f}")
    if recall is not None:
        print(f"  Recall   : {recall:.4f}")
    if f1 is not None:
        print(f"  F1-score : {f1:.4f}")
    if logloss is not None:
        print(f"  Log-loss : {logloss:.4f}")


def main():
    # Load train and test files
    df_train = load_data(TRAIN_PATH)
    df_test  = load_data(TEST_PATH)

    # Determine target column
    target_col = TARGET_COLUMN or detect_target_column(df_train)
    print(f"Using '{target_col}' as target column.\n")

    # Evaluate Random Forests with explicit train/test
    (metrics_std, metrics_w,
     log_std, log_w,
     comp_scores, weights,
     X_train_p, y_train, X_test_p, y_test_enc,
     le, rf, preprocessor) = evaluate_random_forests_explicit(
         df_train, df_test, target_col
    )

    # Report standard RF
    print("Standard Random Forest Performance:")
    print(json.dumps(metrics_std, indent=2))
    print(f"\nStandard RF Log Loss: {log_std:.4f}")
    print_summary(metrics_std, "Standard RF", log_std)

    # Report composite-metric RF
    print("\nComposite-Metric Softmax RF Performance:")
    print(json.dumps(metrics_w, indent=2))
    print(f"\nComposite Softmax RF Log Loss: {log_w:.4f}")
    print_summary(metrics_w, "Composite-Metric RF (softmax)", log_w)

    # Show example composite scores & weights
    print(f"\nComposite Scores (first 10 trees): {np.round(comp_scores[:10],4)}")
    print(f"Weights            (first 10 trees): {np.round(weights[:10],4)}")

    # === Stacked MOE Model Evaluation ===
    experts = [('rf', rf)]
    stacked_moe = StackingClassifier(
        estimators=experts,
        final_estimator=LogisticRegression(),
        cv=5,
        stack_method='predict_proba',
        passthrough=False,
        n_jobs=-1
    )
    stacked_moe.fit(X_train_p, y_train)

    proba_moe = stacked_moe.predict_proba(X_test_p)
    log_moe = log_loss(y_test_enc, proba_moe)
    y_pred_moe = np.argmax(proba_moe, axis=1)
    metrics_moe = classification_report(
        le.inverse_transform(y_test_enc),
        le.inverse_transform(y_pred_moe),
        output_dict=True
    )

    print("\nStacked MOE Model Performance:")
    print(json.dumps(metrics_moe, indent=2))
    print(f"\nStacked MOE Model Log Loss: {log_moe:.4f}")
    print_summary(metrics_moe, "Stacked MOE Model", log_moe)


if __name__ == "__main__":
    main()


Using 'satisfaction' as target column.

Standard Random Forest Performance:
{
  "neutral or dissatisfied": {
    "precision": 0.9569906970473238,
    "recall": 0.9741302408563782,
    "f1-score": 0.9654844084741728,
    "support": 14573.0
  },
  "satisfied": {
    "precision": 0.9661640639023514,
    "recall": 0.9440498114531264,
    "f1-score": 0.9549789310268352,
    "support": 11403.0
  },
  "accuracy": 0.9609254696643055,
  "macro avg": {
    "precision": 0.9615773804748375,
    "recall": 0.9590900261547524,
    "f1-score": 0.960231669750504,
    "support": 25976.0
  },
  "weighted avg": {
    "precision": 0.9610176412361088,
    "recall": 0.9609254696643055,
    "f1-score": 0.9608726915303789,
    "support": 25976.0
  }
}

Standard RF Log Loss: 0.1162

Standard RF Summary:
  Accuracy : 0.9609
  Precision: 0.9610
  Recall   : 0.9609
  F1-score : 0.9609
  Log-loss : 0.1162

Composite-Metric Softmax RF Performance:
{
  "neutral or dissatisfied": {
    "precision": 0.957540164709059,


KeyboardInterrupt: 